### Read the data from the input Excel file

In [2]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
inputFileName = "TransportationPb_Data.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
supplyDF = pd.read_excel(inputFileName, "Supply", skiprows=0, index_col=0)
demandDF = pd.read_excel(inputFileName, "Demand", skiprows=0, index_col=0)
freightDF = pd.read_excel(inputFileName, "Freight", skiprows=0, index_col=2)


### Set the data into dict, easier to manipulate

In [4]:
demandDF

,Demand
Market,
M1,38
M2,129
M3,104
M4,75
M5,126
M6,90
M7,105
M8,91
M9,47


In [5]:
plants = supplyDF.to_dict('index')
markets = demandDF.to_dict('index')
   

freightCost = {}  #freightCost[iPlant][iMarket]
for index, row in freightDF.iterrows():
    if row['Plant'] not in freightCost : freightCost[row['Plant']] = {}
    freightCost[row['Plant']][row['Market']] = row['Cost']
    

In [6]:
# fct obj : min cost st demands are verfied
from pulp import *
prob = LpProblem("Transportation_problem",LpMinimize)

### Add variables


In [7]:
Xvar = LpVariable.dicts("X", ((i,j) for i in plants for j in markets), 
                             lowBound=0, cat='Continuous')
DemandVar= LpVariable.dicts("Demand", (i for i in markets), lowBound=0, cat='Continuous')
CapacityVar= LpVariable.dicts("Capacity", (i for i in plants), lowBound=0, cat='Continuous')


### Add objectives

In [8]:
prob += lpSum([(freightCost[i][j] + plants[i]['Cost'])* Xvar[i,j] for i in plants for j in markets ])

In [9]:
#Notre problème renferme 200 variables Xij, 20 contraintes Capa, 10 contraintes de demand

In [10]:
# for i in plants.keys():
#     for j in markets.keys():
#         print(Xvar[i,j])

### Add constraints

In [11]:
#Capacity constraint (for each plant)
for i in (plants) : 
        prob += lpSum(Xvar[i,j] for j in markets) <= plants[i]['Capacity'] , "capacity[%s]"%(i)



In [12]:
# # #Demand constraint
for j in (markets) :
    prob += lpSum(Xvar[i,j] for i in plants) >= markets[j]['Demand'] , "demand[%s]"%(j)

In [13]:
prob

Transportation_problem:
MINIMIZE
19*X_('P1',_'M1') + 22*X_('P1',_'M10') + 17*X_('P1',_'M2') + 18*X_('P1',_'M3') + 19*X_('P1',_'M4') + 22*X_('P1',_'M5') + 16*X_('P1',_'M6') + 17*X_('P1',_'M7') + 15*X_('P1',_'M8') + 16*X_('P1',_'M9') + 13*X_('P10',_'M1') + 16*X_('P10',_'M10') + 14*X_('P10',_'M2') + 12*X_('P10',_'M3') + 16*X_('P10',_'M4') + 18*X_('P10',_'M5') + 17*X_('P10',_'M6') + 14*X_('P10',_'M7') + 14*X_('P10',_'M8') + 18*X_('P10',_'M9') + 14*X_('P11',_'M1') + 10*X_('P11',_'M10') + 9*X_('P11',_'M2') + 13*X_('P11',_'M3') + 13*X_('P11',_'M4') + 10*X_('P11',_'M5') + 12*X_('P11',_'M6') + 14*X_('P11',_'M7') + 12*X_('P11',_'M8') + 7*X_('P11',_'M9') + 29*X_('P12',_'M1') + 26*X_('P12',_'M10') + 30*X_('P12',_'M2') + 32*X_('P12',_'M3') + 23*X_('P12',_'M4') + 28*X_('P12',_'M5') + 28*X_('P12',_'M6') + 27*X_('P12',_'M7') + 26*X_('P12',_'M8') + 26*X_('P12',_'M9') + 34*X_('P13',_'M1') + 32*X_('P13',_'M10') + 32*X_('P13',_'M2') + 31*X_('P13',_'M3') + 35*X_('P13',_'M4') + 33*X_('P13',_'M5') + 27*X_('P

In [14]:
#prob.writeLP("blenderProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "X" in v.name:
        print(v.name, "=", v.varValue)
        


Status: Optimal
Objective =  16847.0
X_('P1',_'M1') = 0.0
X_('P1',_'M10') = 0.0
X_('P1',_'M2') = 80.0
X_('P1',_'M3') = 0.0
X_('P1',_'M4') = 0.0
X_('P1',_'M5') = 0.0
X_('P1',_'M6') = 0.0
X_('P1',_'M7') = 0.0
X_('P1',_'M8') = 0.0
X_('P1',_'M9') = 0.0
X_('P10',_'M1') = 0.0
X_('P10',_'M10') = 0.0
X_('P10',_'M2') = 0.0
X_('P10',_'M3') = 2.0
X_('P10',_'M4') = 0.0
X_('P10',_'M5') = 0.0
X_('P10',_'M6') = 0.0
X_('P10',_'M7') = 0.0
X_('P10',_'M8') = 0.0
X_('P10',_'M9') = 0.0
X_('P11',_'M1') = 0.0
X_('P11',_'M10') = 0.0
X_('P11',_'M2') = 44.0
X_('P11',_'M3') = 0.0
X_('P11',_'M4') = 0.0
X_('P11',_'M5') = 0.0
X_('P11',_'M6') = 0.0
X_('P11',_'M7') = 0.0
X_('P11',_'M8') = 0.0
X_('P11',_'M9') = 12.0
X_('P12',_'M1') = 0.0
X_('P12',_'M10') = 0.0
X_('P12',_'M2') = 0.0
X_('P12',_'M3') = 0.0
X_('P12',_'M4') = 75.0
X_('P12',_'M5') = 0.0
X_('P12',_'M6') = 0.0
X_('P12',_'M7') = 0.0
X_('P12',_'M8') = 7.0
X_('P12',_'M9') = 8.0
X_('P13',_'M1') = 0.0
X_('P13',_'M10') = 0.0
X_('P13',_'M2') = 0.0
X_('P13',_'M3') = 

Problème Dual 

In [15]:
#Le nombre de variables du pb dual = nb de contraintes du primal
#Le nb de contraintes du pb dual = nb de variables du primal

In [16]:
for name, c in list(prob.constraints.items()):
     if "capacity" in name or "demand" in name:
        print(name, ":", c.pi, "\t\t", c.slack)

capacity_P1_ : -14.0 		 -0.0
capacity_P2_ : -21.0 		 -0.0
capacity_P3_ : 0.0 		 11.0
capacity_P4_ : 0.0 		 -0.0
capacity_P5_ : -10.0 		 -0.0
capacity_P6_ : -20.0 		 -0.0
capacity_P7_ : -23.0 		 -0.0
capacity_P8_ : -9.0 		 -0.0
capacity_P9_ : 0.0 		 53.0
capacity_P10_ : -18.0 		 -0.0
capacity_P11_ : -22.0 		 -0.0
capacity_P12_ : -3.0 		 -0.0
capacity_P13_ : -2.0 		 -0.0
capacity_P14_ : -7.0 		 -0.0
capacity_P15_ : 0.0 		 15.0
capacity_P16_ : -16.0 		 -0.0
capacity_P17_ : -6.0 		 -0.0
capacity_P18_ : -8.0 		 -0.0
capacity_P19_ : -7.0 		 -0.0
capacity_P20_ : -11.0 		 -0.0
demand_M1_ : 29.0 		 -0.0
demand_M2_ : 31.0 		 -0.0
demand_M3_ : 30.0 		 -0.0
demand_M4_ : 26.0 		 -0.0
demand_M5_ : 29.0 		 -0.0
demand_M6_ : 29.0 		 -0.0
demand_M7_ : 29.0 		 -0.0
demand_M8_ : 29.0 		 -0.0
demand_M9_ : 29.0 		 -0.0
demand_M10_ : 29.0 		 -0.0


2-  Why does it make sense to interpret the dual variables associated
 with the Demand constraints as prices

In [17]:
#==> Dans le problème Dual, on adopte une autre stratégie pour voir le problème
#primal : point de vue production :on veut minimiser le cout (transport+production)
#dual : point de vue Gain : on veut maximiser les marges, il s'agit donc de prix

3- Observe that for the obtained price , the obtained flows are such that each supplier is maximising its
margin






In [18]:
#Le modèle cherche à se positionner sur le cout de transport minimum pour chaque supplier
#(le min global du modèle ~ somme des min locaux de chaque supplier)

4- Impose a constraint stating that a supplier shall have not more than 50% of market share on any
market . What is the impact on the prices ?

In [19]:
# # #New constraint
for i in (plants) :
  for j in (markets) :
    prob += Xvar[i,j] <= 0.5 * markets[j]['Demand'],"New_constraint plant[%s] market[%s]"%(i,j)

In [20]:
prob

Transportation_problem:
MINIMIZE
19*X_('P1',_'M1') + 22*X_('P1',_'M10') + 17*X_('P1',_'M2') + 18*X_('P1',_'M3') + 19*X_('P1',_'M4') + 22*X_('P1',_'M5') + 16*X_('P1',_'M6') + 17*X_('P1',_'M7') + 15*X_('P1',_'M8') + 16*X_('P1',_'M9') + 13*X_('P10',_'M1') + 16*X_('P10',_'M10') + 14*X_('P10',_'M2') + 12*X_('P10',_'M3') + 16*X_('P10',_'M4') + 18*X_('P10',_'M5') + 17*X_('P10',_'M6') + 14*X_('P10',_'M7') + 14*X_('P10',_'M8') + 18*X_('P10',_'M9') + 14*X_('P11',_'M1') + 10*X_('P11',_'M10') + 9*X_('P11',_'M2') + 13*X_('P11',_'M3') + 13*X_('P11',_'M4') + 10*X_('P11',_'M5') + 12*X_('P11',_'M6') + 14*X_('P11',_'M7') + 12*X_('P11',_'M8') + 7*X_('P11',_'M9') + 29*X_('P12',_'M1') + 26*X_('P12',_'M10') + 30*X_('P12',_'M2') + 32*X_('P12',_'M3') + 23*X_('P12',_'M4') + 28*X_('P12',_'M5') + 28*X_('P12',_'M6') + 27*X_('P12',_'M7') + 26*X_('P12',_'M8') + 26*X_('P12',_'M9') + 34*X_('P13',_'M1') + 32*X_('P13',_'M10') + 32*X_('P13',_'M2') + 31*X_('P13',_'M3') + 35*X_('P13',_'M4') + 33*X_('P13',_'M5') + 27*X_('P

In [21]:
#prob.writeLP("blenderProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "X" in v.name:
        print(v.name, "=", v.varValue)

Status: Optimal
Objective =  16990.0
X_('P1',_'M1') = 0.0
X_('P1',_'M10') = 0.0
X_('P1',_'M2') = 64.5
X_('P1',_'M3') = 0.0
X_('P1',_'M4') = 0.0
X_('P1',_'M5') = 0.0
X_('P1',_'M6') = 0.0
X_('P1',_'M7') = 0.0
X_('P1',_'M8') = 15.5
X_('P1',_'M9') = 0.0
X_('P10',_'M1') = 0.0
X_('P10',_'M10') = 0.0
X_('P10',_'M2') = 0.0
X_('P10',_'M3') = 2.0
X_('P10',_'M4') = 0.0
X_('P10',_'M5') = 0.0
X_('P10',_'M6') = 0.0
X_('P10',_'M7') = 0.0
X_('P10',_'M8') = 0.0
X_('P10',_'M9') = 0.0
X_('P11',_'M1') = 0.0
X_('P11',_'M10') = 0.0
X_('P11',_'M2') = 52.5
X_('P11',_'M3') = 0.0
X_('P11',_'M4') = 0.0
X_('P11',_'M5') = 0.0
X_('P11',_'M6') = 0.0
X_('P11',_'M7') = 0.0
X_('P11',_'M8') = 0.0
X_('P11',_'M9') = 3.5
X_('P12',_'M1') = 0.0
X_('P12',_'M10') = 0.0
X_('P12',_'M2') = 0.0
X_('P12',_'M3') = 0.0
X_('P12',_'M4') = 37.5
X_('P12',_'M5') = 0.0
X_('P12',_'M6') = 0.0
X_('P12',_'M7') = 7.5
X_('P12',_'M8') = 21.5
X_('P12',_'M9') = 23.5
X_('P13',_'M1') = 0.0
X_('P13',_'M10') = 0.0
X_('P13',_'M2') = 0.0
X_('P13',_'M3') 

In [22]:
for name, c in list(prob.constraints.items()):
     if "capacity" in name or "demand" in name:
        print(name, ":", c.pi, "\t\t", c.slack)

capacity_P1_ : -14.0 		 -0.0
capacity_P2_ : -21.0 		 -0.0
capacity_P3_ : 0.0 		 20.0
capacity_P4_ : 0.0 		 6.0
capacity_P5_ : -10.0 		 -0.0
capacity_P6_ : -18.0 		 -0.0
capacity_P7_ : -23.0 		 -0.0
capacity_P8_ : -9.0 		 -0.0
capacity_P9_ : 0.0 		 53.0
capacity_P10_ : -18.0 		 -0.0
capacity_P11_ : -22.0 		 -0.0
capacity_P12_ : -3.0 		 -0.0
capacity_P13_ : -1.0 		 -0.0
capacity_P14_ : -8.0 		 -0.0
capacity_P15_ : 0.0 		 -0.0
capacity_P16_ : -14.0 		 -0.0
capacity_P17_ : -6.0 		 -0.0
capacity_P18_ : -6.0 		 -0.0
capacity_P19_ : -7.0 		 -0.0
capacity_P20_ : -11.0 		 -0.0
demand_M1_ : 29.0 		 -0.0
demand_M2_ : 31.0 		 -0.0
demand_M3_ : 30.0 		 -0.0
demand_M4_ : 28.0 		 -0.0
demand_M5_ : 29.0 		 -0.0
demand_M6_ : 27.0 		 -0.0
demand_M7_ : 30.0 		 -0.0
demand_M8_ : 29.0 		 -0.0
demand_M9_ : 29.0 		 -0.0
demand_M10_ : 27.0 		 -0.0


In [23]:
#obj = 16847 (old) ==> new : Objective =  16990.0
#demand_M1_ : 29.0 		 -0.0
#demand_M2_ : 31.0 		 -0.0
#demand_M3_ : 30.0 		 -0.0
#demand_M4_ : 26.0 		 -0.0
#demand_M5_ : 29.0 		 -0.0
#demand_M6_ : 29.0 		 -0.0
#demand_M7_ : 29.0 		 -0.0
#demand_M8_ : 29.0 		 -0.0
#demand_M9_ : 29.0 		 -0.0
#demand_M10_ : 29.0 

#Dual Problem

In [24]:
B=[markets[j]['Demand'] for j in markets]+[plants[i]['Capacity'] for i in plants]
print(B)


[38, 129, 104, 75, 126, 90, 105, 91, 47, 61, 80, 9, 22, 69, 5, 81, 37, 18, 53, 2, 56, 90, 67, 45, 56, 49, 1, 96, 83, 26]


In [25]:
import numpy as np

A=np.zeros((30,230))
k=0
for i in range(10) :
        for j in range(20):
            A[i,j*10+k]=1.0
        k+=1
        
for i in range(10) :
    A[i,i+200]=-1.

for i in range(20) :
        for j in range(10):
            A[i+10,i*10+j]=1.0
        
for i in range(20) :
    A[i+10,i+210]=1.0
print(A)
tA=A.transpose()

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [26]:
C=[freightCost[i][j] + plants[i]['Cost'] for i in plants for j in markets ]+[0]*30
print(C)

[19, 17, 18, 19, 22, 16, 17, 15, 16, 22, 17, 17, 9, 18, 10, 12, 13, 15, 10, 11, 30, 34, 36, 30, 29, 37, 33, 34, 31, 29, 31, 34, 36, 36, 29, 31, 32, 36, 33, 34, 27, 21, 25, 22, 21, 22, 22, 29, 26, 21, 15, 17, 17, 17, 12, 9, 13, 12, 11, 9, 6, 12, 14, 15, 11, 14, 12, 6, 14, 10, 26, 23, 24, 26, 20, 21, 27, 26, 28, 24, 35, 33, 31, 30, 35, 33, 35, 30, 36, 33, 13, 14, 12, 16, 18, 17, 14, 14, 18, 16, 14, 9, 13, 13, 10, 12, 14, 12, 7, 10, 29, 30, 32, 23, 28, 28, 27, 26, 26, 26, 34, 32, 31, 35, 33, 27, 27, 30, 28, 32, 27, 27, 23, 20, 28, 22, 22, 25, 23, 24, 29, 35, 33, 32, 29, 32, 33, 32, 29, 33, 19, 17, 21, 17, 17, 13, 21, 17, 19, 13, 28, 25, 26, 24, 28, 29, 31, 23, 24, 26, 23, 27, 22, 23, 24, 21, 24, 23, 24, 26, 24, 24, 23, 22, 23, 28, 28, 22, 29, 29, 21, 25, 24, 18, 19, 22, 19, 25, 18, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [27]:
prob2 = LpProblem("Transportation_problem",LpMinimize)

In [28]:
Xvar2=LpVariable.dict("dual", (i for i in range(230)), lowBound=0, cat='Continuous')
print(Xvar2[1])

dual_1


In [29]:
prob2+= lpSum(Xvar2[i]*C[i] for i in Xvar2.keys())
for i in range(len(A)) :
    prob2+= lpSum(Xvar2[j]*A[i][j] for j in range(len(A[i])))==B[i],"constraint [%s]]"%(i)

In [30]:
print(prob2)

Transportation_problem:
MINIMIZE
19*dual_0 + 17*dual_1 + 17*dual_10 + 14*dual_100 + 9*dual_101 + 13*dual_102 + 13*dual_103 + 10*dual_104 + 12*dual_105 + 14*dual_106 + 12*dual_107 + 7*dual_108 + 10*dual_109 + 17*dual_11 + 29*dual_110 + 30*dual_111 + 32*dual_112 + 23*dual_113 + 28*dual_114 + 28*dual_115 + 27*dual_116 + 26*dual_117 + 26*dual_118 + 26*dual_119 + 9*dual_12 + 34*dual_120 + 32*dual_121 + 31*dual_122 + 35*dual_123 + 33*dual_124 + 27*dual_125 + 27*dual_126 + 30*dual_127 + 28*dual_128 + 32*dual_129 + 18*dual_13 + 27*dual_130 + 27*dual_131 + 23*dual_132 + 20*dual_133 + 28*dual_134 + 22*dual_135 + 22*dual_136 + 25*dual_137 + 23*dual_138 + 24*dual_139 + 10*dual_14 + 29*dual_140 + 35*dual_141 + 33*dual_142 + 32*dual_143 + 29*dual_144 + 32*dual_145 + 33*dual_146 + 32*dual_147 + 29*dual_148 + 33*dual_149 + 12*dual_15 + 19*dual_150 + 17*dual_151 + 21*dual_152 + 17*dual_153 + 17*dual_154 + 13*dual_155 + 21*dual_156 + 17*dual_157 + 19*dual_158 + 13*dual_159 + 13*dual_16 + 28*dual_160 + 2

In [31]:
#prob.writeLP("blenderProblem.lp", writeSOS=1, mip=1)
prob2.solve()
print("Status:", LpStatus[prob2.status])
print ("Objective = ", value(prob2.objective))
varsDict = {}

for v in prob2.variables():
    varsDict[v.name] = v.varValue
    if "dual" in v.name:
        print(v.name, "=", v.varValue)

Status: Optimal
Objective =  16847.0
dual_0 = 0.0
dual_1 = 0.0
dual_10 = 0.0
dual_100 = 0.0
dual_101 = 41.0
dual_102 = 0.0
dual_103 = 0.0
dual_104 = 0.0
dual_105 = 0.0
dual_106 = 0.0
dual_107 = 0.0
dual_108 = 15.0
dual_109 = 0.0
dual_11 = 0.0
dual_110 = 0.0
dual_111 = 0.0
dual_112 = 0.0
dual_113 = 75.0
dual_114 = 0.0
dual_115 = 0.0
dual_116 = 0.0
dual_117 = 10.0
dual_118 = 0.0
dual_119 = 5.0
dual_12 = 9.0
dual_120 = 0.0
dual_121 = 0.0
dual_122 = 0.0
dual_123 = 0.0
dual_124 = 0.0
dual_125 = 7.0
dual_126 = 60.0
dual_127 = 0.0
dual_128 = 0.0
dual_129 = 0.0
dual_13 = 0.0
dual_130 = 0.0
dual_131 = 0.0
dual_132 = 0.0
dual_133 = 0.0
dual_134 = 0.0
dual_135 = 0.0
dual_136 = 45.0
dual_137 = 0.0
dual_138 = 0.0
dual_139 = 0.0
dual_14 = 0.0
dual_140 = 1.0
dual_141 = 0.0
dual_142 = 0.0
dual_143 = 0.0
dual_144 = 49.0
dual_145 = 0.0
dual_146 = 0.0
dual_147 = 0.0
dual_148 = 6.0
dual_149 = 0.0
dual_15 = 0.0
dual_150 = 0.0
dual_151 = 0.0
dual_152 = 0.0
dual_153 = 0.0
dual_154 = 0.0
dual_155 = 49.0
dual_

In [33]:
value(prob2.objective)

16847.0